In [1]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

#Setup Paths
import os
BASE_DIR = '/content/drive/MyDrive/speech_understanding_project'
DATA_DIR = os.path.join(BASE_DIR, 'data')
os.makedirs(DATA_DIR, exist_ok=True)


Mounted at /content/drive


In [2]:
# Install Required Packages
!pip install datasets torchaudio librosa soundfile tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install datasets


In [ ]:
from huggingface_hub import login

# Replace 'YOUR_TOKEN' with your actual Hugging Face API token
login(token='#Your Token')

# 📍 Load Common Voice for English (HRL) and Swahili (LRL)
from datasets import load_dataset

# Pick one HRL and one LRL language
HRL_LANG = "en"       # English
LRL_LANG = "hi"       # Hindi

# Load 50 samples each (for demo — increase as needed)
hrl_dataset = load_dataset("mozilla-foundation/common_voice_13_0", HRL_LANG, split="test", streaming=True).take(50)
lrl_dataset = load_dataset("mozilla-foundation/common_voice_13_0", LRL_LANG, split="test", streaming=True).take(50)


In [6]:
# 📍 Preprocess and Save WAV Files
import librosa
import soundfile as sf
from tqdm import tqdm

def save_commonvoice_wavs(dataset, output_dir, prefix):
    os.makedirs(output_dir, exist_ok=True)
    paths = []
    for i, item in enumerate(tqdm(dataset)):
        try:
            audio = item["audio"]
            y = audio["array"]
            sr = audio["sampling_rate"]

            # Resample to 16kHz
            if sr != 16000:
                y = librosa.resample(y, orig_sr=sr, target_sr=16000)

            # Normalize
            y = y / max(abs(y)) if max(abs(y)) > 0 else y
            filename = f"{prefix}_{i:03d}.wav"
            filepath = os.path.join(output_dir, filename)
            sf.write(filepath, y, 16000)
            paths.append((filepath, librosa.get_duration(y=y, sr=16000)))
        except Exception as e:
            print(f"Skipping sample {i}: {e}")
    return paths

# Save both HRL and LRL
hrl_dir = os.path.join(DATA_DIR, 'commonvoice_hrl')
lrl_dir = os.path.join(DATA_DIR, 'commonvoice_lrl')

hrl_manifest = save_commonvoice_wavs(hrl_dataset, hrl_dir, "hrl")
lrl_manifest = save_commonvoice_wavs(lrl_dataset, lrl_dir, "lrl")


0it [00:00, ?it/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 1it [00:00,  2.20it/s]
Reading metadata...: 16372it [00:00, 25208.28it/s]
50it [00:23,  2.15it/s]
0it [00:00, ?it/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 2947it [00:00, 7129.19it/s]
50it [00:03, 13.39it/s]


In [7]:
# 📍 Save Manifests
def write_manifest(pairs, path):
    with open(path, 'w') as f:
        for p, dur in pairs:
            f.write(f"{p}\t{dur:.2f}\n")
    print(f"Manifest written to {path}")

write_manifest(hrl_manifest, os.path.join(DATA_DIR, 'hrl_manifest.tsv'))
write_manifest(lrl_manifest, os.path.join(DATA_DIR, 'lrl_manifest.tsv'))


Manifest written to /content/drive/MyDrive/speech_understanding_project/data/hrl_manifest.tsv
Manifest written to /content/drive/MyDrive/speech_understanding_project/data/lrl_manifest.tsv
